# 5 SVM 课堂练习

姓名：华勤达

学号：2018212212229

贷款违约数据集来自Kaggle数据科学竞赛平台2011年的一个竞赛项目“Give Me Some Cerdit”，该数据集共包含25000个样本，其中150000个样本作为训练集，100000个样本用作测试集。

该训练集共有150000条借款人的历史数据，其中违约样本10026条，占样本总量的6.684%，贷款违约率为6.684%，未违约样本139974条，占样本总量的93.316%。可以看出该数据集是一个典型的高度不平衡的数据。数据集中包括了借款人的年龄、收入、家庭等情况和贷款情况，共11个变量，其中SeriousDlqin2yrs是label标签，另外10个变量为预测特征。下表列出了变量名称及数据类型情况： 

表1 数据集变量情况

| 变量名 | 变量描述 | 类型 |
|  - | -  | -|
|SeriousDlqin2yrs | 是否违约 | Y/N|
|RevolvingUtilizationOfUnsecuredLines | 信用卡和个人信用贷款的总额（不含房贷、类似车贷的分期付款等）除以授信额度之和 | 百分比 |
|age | 借款人年龄 | 整数 |
|NumberOfTime30-59DaysPastDueNotWorse | 过去两年中借款人逾期30-59天的次数 | 整数 |
|DebtRatio | 每月债务偿还数、赡养费、生活成本等除以每月总收入 | 百分比 |
|MonthlyIncome | 月收入 | 实数|
|NumberOfOpenCreditLinesAndLoans | 开放贷款（Open loans，分期付款如车贷和抵押）和信用贷款（Lines of credit,如信用卡）的数量 |整数 |
|NumberOfTimes90DaysLate| 过去两年中借款人逾期90天及以上的次数 | 整数|
|NumberRealEstateLoansOrLines | 抵押和房地产贷款含房屋抵押式信用贷款的次数 | 整数|
|NumberOfTime60-89DaysPastDueNotWorse| 过去两年中借款人逾期60-89天的次数 | 整数 |
|NumberOfDependents | 不包括本人在内家庭中需要抚养的人（配偶及子女等等）数 | 整数 |

1. 数据导入和清洗

In [1]:
import pandas as pd

train_data = pd.read_csv('cs-training(2).csv')

feature_names = ['个人信用贷款/授信额度','借款人年龄','逾期30-59天次数','债务收入比','贷款数量']
drop_features = ['Unnamed: 0','MonthlyIncome','NumberOfTimes90DaysLate','NumberRealEstateLoansOrLines',
                 'NumberOfTime60-89DaysPastDueNotWorse','NumberOfDependents']
target_names = ['正常','违约']

train_data = train_data.drop(drop_features, axis=1).dropna()
X = train_data.drop(["SeriousDlqin2yrs"], axis=1).copy()
y = train_data.SeriousDlqin2yrs
X.shape, y.shape

((150000, 5), (150000,))

2. 分训练集和测试集

In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
                                   X, y, test_size=0.2, random_state=42)
X_train.shape,X_test.shape

((120000, 5), (30000, 5))

In [35]:
from sklearn.svm import SVC
my_svc = SVC(kernel='rbf', C=1, gamma=0.5, cache_size=200)

# 构建pipeline
pipe_svc = Pipeline([('sc', StandardScaler()),
                    ('my_svc',my_svc)
                    ])

In [36]:
# print()
pipe_svc.fit(X_train, y_train)

Pipeline(steps=[('sc', StandardScaler()), ('my_svc', SVC(C=1, gamma=0.5))])

3.采用Pipeline进行数据归一化和训练模型,并且不设置class_weight参数

In [37]:
#检查模型准确率
from sklearn.metrics import accuracy_score
%time y_pred = pipe_svc.predict(X_test)
accuracy_score(y_test, y_pred)

CPU times: user 54.1 s, sys: 260 ms, total: 54.4 s
Wall time: 54.7 s


0.9352666666666667

In [38]:
#查看混淆矩阵
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[28035,     9],
       [ 1933,    23]])

In [39]:
#查看F1score
from sklearn import metrics
score = metrics.f1_score(y_test,y_pred)
score

0.02313883299798793

4. 再次定义 Pipline模型，并设置class_weight = ?

In [41]:
my_svc_we = SVC(kernel = 'rbf', C = 1, gamma = 0.5, cache_size = 200, class_weight = {0:1,1:9})

# 构建pipeline
pipe_svc2 = Pipeline([('sc', StandardScaler()),
                    ('my_svc_we', my_svc_we)
                    ])
%time pipe_svc2.fit(X_train, y_train)

CPU times: user 9min 38s, sys: 3.12 s, total: 9min 41s
Wall time: 9min 44s


Pipeline(steps=[('sc', StandardScaler()),
                ('my_svc_we', SVC(C=1, class_weight={0: 1, 1: 9}, gamma=0.5))])

In [42]:
#检查模型准确率
from sklearn.metrics import accuracy_score

%time y_pred = pipe_svc2.predict(X_test)
accuracy_score(y_test, y_pred)

CPU times: user 2min 45s, sys: 944 ms, total: 2min 46s
Wall time: 2min 48s


0.8363666666666667

In [43]:
#查看混淆矩阵
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[24042,  4002],
       [  907,  1049]])

In [44]:
#查看F1score
from sklearn import metrics

score = metrics.f1_score(y_test,y_pred)
score

0.2994148708434423